<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/twadr_5_mixed_pubmed_bigtweet_twadr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-5.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-5.validation.csv",header=None)
valid.head()

,0,1
0,13,Insomnia
1,13,awake in the middle of the night
2,57,hypertrophic scarring
3,85,tremor
4,94,anxiety


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-5.test.csv",header=None)
test.head()

,0,1
0,13,up all night
1,13,insomnia
2,93,shooting
3,133,night terrors
4,231,lucid dreamer


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_twadr.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'was',
 'diclofenac',
 'were',
 'on',
 'is',
 'pain',
 'rt',
 'sodium',
 'by',
 'drug',
 'that',
 'as',
 'i',
 'at',
 'or',
 'patients',
 'this',
 'from',
 'you',
 'disease',
 'be',
 'increased',
 'mg',
 'study',
 'group',
 'my',
 'mood',
 'are',
 'not',
 'treatment',
 'it',
 'after',
 'me',
 'an',
 'have',
 'amp',
 'p',
 'release',
 'gluten',
 'disorder',
 ',',
 'all',
 'effect',
 'we',
 'withdrawal',
 'free',
 'abnormal',
 'than',
 'symptoms',
 'no',
 'decreased',
 'weight',
 'compared',
 'out',
 'more',
 'your',
 'anxiety',
 'new',
 'day',
 'effects',
 'using',
 'groups',
 'our',
 'time',
 'up',
 'both',
 'two',
 'about',
 'h',
 'significantly',
 'drugs',
 'blood',
 'like',
 'inflammatory',
 'its',
 'these',
 'skin',
 'between',
 'acute',
 'but',
 'had',
 'can',
 'disorders',
 'potassium',
 'significant',
 'anti',
 'has',
 'use',
 'be

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.725287,4.212960,0.341787,23:17


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.996613,3.832397,0.377547,23:11


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.728415,3.788286,0.382346,23:10


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.667167,3.756520,0.385819,23:41


In [25]:
learn.save_encoder('pubmed_cadec_twadrl_mixed_fold5_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_cadec_twadrl_mixed_fold5_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.168226,6.798484,0.061538,00:24


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.193985,4.988405,0.161538,00:28


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.642177,4.817461,0.184615,01:06


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.433926,4.474031,0.215385,01:07
1,5.174873,4.284108,0.230769,01:11


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.887179,4.025888,0.300000,01:11
1,4.731250,3.997002,0.269231,01:06


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.473302,3.916573,0.338462,01:06
1,4.306895,3.909714,0.315385,01:11


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.117854,3.747685,0.330769,01:07
1,3.974382,3.817488,0.323077,01:11


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.818016,3.679688,0.330769,01:09
1,3.690136,3.670678,0.346154,01:08


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.474772,3.658468,0.330769,01:10
1,3.417334,3.635795,0.315385,01:05
2,3.240014,3.628785,0.361538,01:06
3,3.090044,3.603916,0.353846,01:10


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.871830,3.675026,0.346154,01:14
1,2.824067,3.680312,0.338462,01:08
2,2.643530,3.715038,0.353846,01:10
3,2.500571,3.659269,0.361538,01:11


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.350294,3.614859,0.376923,01:10
1,2.320533,3.717751,0.346154,01:10
2,2.138309,3.761787,0.338462,01:12
3,1.963968,3.751354,0.338462,01:13


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.833150,3.794940,0.369231,01:10
1,1.821217,3.821647,0.361538,01:12
2,1.677437,3.872115,0.361538,01:12
3,1.569198,3.939014,0.369231,01:06


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.410509,3.941470,0.346154,01:09
1,1.402094,4.114700,0.323077,01:09
2,1.302130,4.068210,0.346154,01:06
3,1.192146,4.126517,0.361538,01:10


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.120922,4.063375,0.369231,01:09
1,1.106292,4.214025,0.346154,01:13
2,1.019414,4.195822,0.361538,01:12
3,0.919003,4.202745,0.369231,01:10


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.878306,4.293581,0.330769,01:07
1,0.910996,4.427607,0.315385,01:08
2,0.834803,4.661834,0.323077,01:09
3,0.740528,4.539474,0.330769,01:09


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.692299,4.419730,0.346154,01:10
1,0.686945,4.534122,0.353846,01:07


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.679308,4.542460,0.315385,01:15
1,0.665630,4.701327,0.369231,01:05


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.623675,4.640868,0.369231,01:07


In [44]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.578105,4.739814,0.346154,01:10


In [45]:
# save the best model

learn.save_encoder('pubmed_cadec_twadrl_mixed_fold5')

# Part three: Predict on the test dataset

In [46]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [47]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[1099, 1787, 1145, 133, 231, 233, 233, 249, 300, 1582, 1136, 1787, 824, 532, 550, 1373, 582, 749, 644, 744, 744, 749, 749, 1597, 1597, 1597, 974, 792, 792, 813, 1578, 1321, 1308, 1096, 1278, 858, 1815, 974, 974, 974, 978, 978, 978, 978, 978, 978, 1007, 1019, 107, 824, 1099, 1545, 1356, 1136, 1373, 1432, 1192, 561, 1227, 1227, 1295, 1295, 1295, 1308, 1340, 441, 947, 393, 824, 1884, 1884, 809, 792, 1341, 1508, 1446, 224, 1701, 1701, 1610, 1973, 1815, 1594, 1356, 1597, 1597, 852, 1610, 1610, 1610, 1610, 300, 1145, 1048, 1099, 749, 1711, 824, 1718, 1767, 1727, 1718, 2026, 1767, 1669, 1767, 1727, 1718, 1073, 1099, 1787, 2191, 1727, 393, 1787, 2119, 1787, 923, 1815, 1815, 1822, 393, 2147, 1936, 1936, 1942, 1529, 1341, 1798, 1599, 858, 1295, 2032, 1787, 189, 2081, 2103, 1529, 1787, 233, 2032, 1529, 1936]
[13, 13, 93, 133, 231, 233, 233, 249, 300, 362, 396, 418, 441, 532, 550, 582, 582, 596, 644, 744, 745, 749, 749, 791, 791, 791, 792, 792, 792, 813, 815, 818, 822, 852, 857, 858, 974, 

In [48]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

61
0.42657342657342656
